Pretty

In [1]:
import pandas as pd
import json
import bcrypt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [7]:
posts = pd.read_json('posts.json', lines=True)

posts.to_json('posts_pretty.json', orient='records', lines=False, indent=4)

In [6]:
users = pd.read_json('users.json', lines=True)

users.to_json('users_pretty.json', orient='records', lines=False, indent=4)

In [9]:
ingredients = pd.read_json('ingredients.json', lines=True)

ingredients.to_json('ingredients_pretty.json', orient='records', lines=False, indent=4)

Modifica id degli user nei post

In [11]:
# Leggi il file JSON dei post
with open('posts_pretty.json', 'r') as file:
    posts = json.load(file)
with open('users_pretty.json', 'r') as file:
    users = json.load(file)



# Crea una mappa dei commenti basata su user_id
users_map = {}
for user in users:
    user_id = user['id']
    if user_id not in users_map:
        users_map[user_id] = []
    users_map[user_id].append(user)

# Aggiungi i commenti ai post corrispondenti
for post in posts:
    user_id = post['idUser']
    post['idUser'] = users_map.get(user_id, [])[0]["_id"]
    for comment in post["comments"]:
        user_id = comment['idUser']
        comment['idUser'] = users_map.get(user_id, [])[0]["_id"]
    for star in post["starRankings"]:
        user_id = star['idUser']
        star['idUser'] = users_map.get(user_id, [])[0]["_id"]        

# Salva o restituisci il nuovo JSON
with open('posts_updated.json', 'w') as file:
    json.dump(posts, file, indent=4)

In [13]:
# Leggi il file JSON dei post
with open('posts_pretty.json', 'r') as file:
    posts = json.load(file)
with open('users_pretty.json', 'r') as file:
    users = json.load(file)

# Crea una mappa dei commenti basata su user_id
posts_map = {}
for post in posts:
    post_id = post['id']
    if post_id not in posts_map:
        posts_map[post_id] = []
    posts_map[post_id].append(post)

# Aggiungi i commenti ai post corrispondenti
for user in users:
    for post in user["posts"]:
        post_id = post['idPost']
        post['idPost'] = posts_map.get(post_id, [])[0]["_id"]    

# Salva o restituisci il nuovo JSON
with open('users_updated.json', 'w') as file:
    json.dump(users, file, indent=4)

Puliamo

In [14]:
with open('users_updated.json', 'r') as file:
    users = json.load(file)
    
for user in users:
    del user["id"]
        
with open('users_database.json', 'w') as file:
    json.dump(users, file, indent=4)

In [2]:
with open('users_database.json', 'r') as file:
    users = json.load(file)
    
for user in users:
    #string to bytes
    password = user["password"].encode()
    salt = bcrypt.gensalt()  # Genera un sale
    hashed = bcrypt.hashpw(password, salt)
    user["password"] = hashed.decode()

        
with open('users_database_hashed.json', 'w') as file:
    json.dump(users, file, indent=4)

In [16]:
with open('users_database_hashed.json', 'r') as file:
    users_hashed = json.load(file)
with open('users_database.json', 'r') as file:
    users = json.load(file)

# Verifica delle password

id = 27900

input_password = users[id]["password"]  # Sostituisci con la password effettiva
stored_hash = users_hashed[id]["password"].encode()  # Ottieni l'hash salvato e codificalo in bytes

if bcrypt.checkpw(input_password.encode(), stored_hash):
    print(f"Password corretta per l'utente {users_hashed[id]['username']}")
else:
    print(f"Password errata per l'utente {users_hashed[id]['username']}")

27901
Password corretta per l'utente mathew_pierce_34


In [3]:
with open('posts_updated.json', 'r') as file:
    posts = json.load(file)
    
for post in posts:
    del post["id"]
    if post["description"] == None:
        del post["description"]
    if post["avgStarRanking"] == "No votes available":
        del post["avgStarRanking"]
    else:
        post["avgStarRanking"] = float(post["avgStarRanking"])
    if post["comments"] == []:
        del post["comments"]
    if post["starRankings"] == []:
        del post["starRankings"]
        
with open('posts_database.json', 'w') as file:
    json.dump(posts, file, indent=4)